# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [7]:
# Write your code below.
%load_ext dotenv
%dotenv



The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [8]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [10]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)



For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [13]:
# Write your code below.

dd_px = dd.read_parquet(parquet_files).set_index("Ticker")

# adding lags for variable Close
dd_lags_close = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))  
) #For each ticker, we are adding a column called Close_lag_1 which is the previous day's Close price
# adding lags for variable Adj Close
dd_lags_AdjClose = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(AdjClose_lag_1 = x['Adj Close'].shift(1))  
) #prev day Adj Close price for each ticker
# adding returns for variable Close
dd_rets1 = dd_lags_close.assign(
    Close_returns = lambda x: x['Close'] / x['Close_lag_1'] - 1  
) #returns for each row in the Close column
# adding returns for variable Adj Close

dd_rets2 = dd_lags_AdjClose.assign(
    Adj_Close_returns = lambda x: x['Adj Close'] / x['AdjClose_lag_1'] - 1  
) #same thing but for Adj Close
# adding hi_lo_range
dd_lags_hi_lo = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(hi_lo_range = x['High'] - x['Low'])  
) #hi-low range for each ticker

dd_feat = dd.concat([dd_rets1, dd_rets2[['Adj_Close_returns']], dd_lags_hi_lo[['hi_lo_range']]], axis=1)

/var/folders/ct/7p_jm4ss0f70mkwznhhwhj340000gn/T/ipykernel_25344/2873430075.py:6: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_lags_close = dd_px.groupby('Ticker', group_keys=False).apply(
/var/folders/ct/7p_jm4ss0f70mkwznhhwhj340000gn/T/ipykernel_25344/2873430075.py:10: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_lags_AdjClose = dd_px.groupby('Ticker', group_keys=False).apply(
/var/folders/ct/7p_jm4ss0f70mkwznhhwhj340000gn/T/ipykernel_25344/2873430075.py:23: UserWarning: `meta` is not specified, inferred fr

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [14]:
# Write your code below.

import dask.dataframe as dd

# Function to apply rolling mean on each partition
def rolling_mean_partition(df):
    # Ensure that 'returns' column exists before applying rolling
    df['returns'] = df['Close_returns'].combine_first(df['Adj_Close_returns'])
    
    # Apply rolling mean on the 'returns' column
    df['returns_ma'] = df['returns'].rolling(10, min_periods=1).mean()
    return df

# Apply the function to each partition using map_partitions
dd_feat = dd_feat.map_partitions(rolling_mean_partition)

# Compute the final DataFrame
dd_feat_pd = dd_feat.compute()

# Print the result
print(dd_feat_pd.head())



/opt/anaconda3/envs/dsi_participant/lib/python3.9/site-packages/dask_expr/_concat.py:146: UserWarning: Concatenating dataframes with unknown divisions.
We're assuming that the indices of each dataframes are 
 aligned. This assumption is not generally safe.
  warnings.warn(


Price                       Date  Adj Close      Close       High        Low  \
Ticker                                                                         
DOV    2017-01-03 00:00:00+00:00  53.950459  61.728596  62.625202  60.920841   
DOV    2017-01-04 00:00:00+00:00  54.204605  62.019386  62.075928  61.445881   
DOV    2017-01-05 00:00:00+00:00  54.028111  61.817448  62.431339  61.074314   
DOV    2017-01-06 00:00:00+00:00  54.868214  62.778675  63.780293  62.390953   
DOV    2017-01-09 00:00:00+00:00  54.162254  61.970921  62.924072  61.744751   

Price        Open     Volume  Year  Close_lag_1  Close_returns  \
Ticker                                                           
DOV     61.639744  1637998.0  2017          NaN            NaN   
DOV     61.882069  1165948.0  2017    61.728596       0.004711   
DOV     61.793217  1155178.0  2017    62.019386      -0.003256   
DOV     62.463650  3123350.0  2017    61.817448       0.015549   
DOV     62.705978  1271055.0  2017    62.77

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return? - No. I created similar features in the previous step. Pandas make sense when we are working with a smaller dataset. Or to make the code easier to read for other colleagues.
+ Would it have been better to do it in Dask? Why? - It took me more than 2 minutes to create these features in pandas, it took only a fraction of a second in dask. Computationally the features are similar. Dask is faster because it runs in parallel. But pandas is slower. But pandas code is easier to read.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.